In [1]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-5-nano")

llm.invoke("hi")

AIMessage(content='Hi there! How can I help today? I can answer questions, explain something, write or edit text, brainstorm ideas, or assist with tasks. What would you like to do?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 238, 'prompt_tokens': 7, 'total_tokens': 245, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D085dPnxxV2GXcfKpIhrJo5Nq1AFP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bdc0e-129f-7ee3-9faa-efd2774ec7c3-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 7, 'output_tokens': 238, 'total_tokens': 245, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio':

In [15]:
# Initialize tool calls
from langchain_core.tools import tool
from typing import Literal
from datetime import datetime
from pydantic import BaseModel

@tool
def write_email(to:str, subject:str, thread:str)->str:
    """Write and send email"""
    return f"Email has been send to user : {to}"

@tool
def schedule_meeting(attendies:list[str], subject:str, duration_min:int, preferred_day:datetime, start_time:str)->str:
    """Schedule a calender meeting"""
    date_str = preferred_day.strftime("%A, %B %d, %Y")
    return f"Meeting '{subject}' scheduled on {date_str} at {start_time} for {duration_minutes} minutes with {len(attendees)} attendees"

@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

@tool
class Done(BaseModel):
    """Email has been send"""
    done:bool
    

In [17]:
from langgraph.graph import MessagesState

class State(MessagesState):
    email_input : dict
    classification_decision: Literal["ignore", "notify", "respond"]


In [19]:
from pydantic import Field
class RouterSchema(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

In [31]:
tools = [write_email, schedule_meeting, check_calendar_availability, Done]
tools_by_name = {tool.name: tool for tool in tools}

tools_by_name

{'write_email': StructuredTool(name='write_email', description='Write and send email', args_schema=<class 'langchain_core.utils.pydantic.write_email'>, func=<function write_email at 0x12ce3c860>),
 'schedule_meeting': StructuredTool(name='schedule_meeting', description='Schedule a calender meeting', args_schema=<class 'langchain_core.utils.pydantic.schedule_meeting'>, func=<function schedule_meeting at 0x12ce3ccc0>),
 'check_calendar_availability': StructuredTool(name='check_calendar_availability', description='Check calendar availability for a given day.', args_schema=<class 'langchain_core.utils.pydantic.check_calendar_availability'>, func=<function check_calendar_availability at 0x12ce3c9a0>),
 'Done': StructuredTool(name='Done', description='Email has been send', args_schema=<class 'langchain_core.utils.pydantic.Done'>, func=<class '__main__.Done'>)}

In [35]:
from operator import itemgetter
from langgraph.graph import END
from langgraph.types import Command

llm_with_tools = llm.bind_tools(tools,  tool_choice="any")
llm_router = llm.with_structured_output(RouterSchema) 


def get_user_prompt(sender:str, receiver:str, subject:str, thread:str)->str:
    return  f"""Classify the email

        Sender : {sender}

        Receiver : {receiver}

        Subject : {subject}

        Thread : {thread}

    """


def traige_router(state:State)-> Command[Literal["response_agent", "__end__"]]:
    email_input = state["email_input"]
    # destructure the email
    sender, receiver, subject, thread = itemgetter("from", "to", "subject", "thread")(email_input)
    # create user prompt
    u_prompt = get_user_prompt(sender=sender, receiver=receiver, subject=subject, thread=thread)
    ## invoke the llm
    response = llm_router.invoke(u_prompt)

    if response.classification == "ignore":
        goto = END
        update = {
            "classification_decison" : "ignore"
        }
    elif response.classification == "notify":
        goto=END
        update = {
            "classification_decision" : "notify"
        }
    elif response.classification == "respond":
        goto= "response_agent"
        update = {
            "classification_decision" : "respond",
            "messages": {
                "role": "user",
                "content": u_prompt
            }
        }
    else:
        return ValueError("Invalid value")
    
    return Command(goto=goto, update=update)
    
    

In [36]:
def call_llm(state:State):
    """LLM decide whther to call the tools or not"""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": response}

In [49]:
from langgraph.graph import StateGraph, START, END

agent = StateGraph(State)
agent.add_node("call_llm", call_llm)
agent.add_edge(START, "call_llm")

agent = agent.compile()

In [72]:
email_input = {
  "from": "Alice Smith <alice.smith@company.com>",
  "to": "John Doe <john.doe@company.com>",
  "subject": "Quick question about API documentation",
  "thread": "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"
}

In [73]:
from langgraph.graph import StateGraph, START, END

test = StateGraph(State)
test.add_node("triage", traige_router)
test.add_node("response_agent", agent)
test.add_edge(START, "triage")


test = test.compile()

In [74]:
email_input

{'from': 'Alice Smith <alice.smith@company.com>',
 'to': 'John Doe <john.doe@company.com>',
 'subject': 'Quick question about API documentation',
 'thread': "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"}

In [75]:
outs = test.invoke({"email_input": email_input})
outs

{'messages': [HumanMessage(content="Classify the email\n\n        Sender : Alice Smith <alice.smith@company.com>\n\n        Receiver : John Doe <john.doe@company.com>\n\n        Subject : Quick question about API documentation\n\n        Thread : Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice\n\n    ", additional_kwargs={}, response_metadata={}, id='af431fb9-edf7-48cd-9a3e-a88d0e453ee9'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1337, 'prompt_tokens': 333, 'total_tokens': 1670, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1216, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_token

In [77]:
outs["messages"][-1].tool_calls

[{'name': 'write_email',
  'args': {'to': 'docs-team@company.com',
   'subject': 'Classification: Email from Alice Smith about API documentation',
   'thread': 'Classification: This email from Alice Smith to John Doe is an inquiry about API documentation. It requests clarification on missing endpoints in the new authentication service specs and asks whether /auth/refresh and /auth/validate were unintentionally omitted or should be updated in the docs. Suggested category: Inquisitive clarification request / gap in API docs. Primary action: Confirm whether endpoints exist and update docs accordingly.'},
  'id': 'call_bXx0UxfyycfzK3KeMMMIE3bM',
  'type': 'tool_call'}]